In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing

In [3]:
names = ['id','thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit','class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/' +
                 'breast-cancer-wisconsin/breast-cancer-wisconsin.data',
                names=names,na_values='?',header=None)
df = df.dropna()
df.head(6)

# 2 is benign, 4 is malignant

,id,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
5,1017122,8,10,10,8,7,10.0,9,7,1,4


In [4]:
df['class']=df['class'].map({4:1,2:0})
df.head(5)

,id,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,1000025,5,1,1,1,2,1.0,3,1,1,0
1,1002945,5,4,4,5,7,10.0,3,2,1,0
2,1015425,3,1,1,1,2,2.0,3,1,1,0
3,1016277,6,8,8,1,3,4.0,3,7,1,0
4,1017023,4,1,1,3,2,1.0,3,1,1,0


In [5]:
df = df.astype(int)
df.head(5)

,id,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [7]:
xnames = ['thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit']
x = np.array(df[xnames])

y=df['class'].values

In [8]:
#splitting the input into training and testing

from sklearn.model_selection import train_test_split

xtr, xts, ytr, yts = train_test_split(x,y, test_size=0.2)

In [10]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtr = sc.fit_transform(xtr)
xts = sc.transform(xts)

In [11]:
#NEURAL NET MODEL

import os
import tensorflow as tf
import tensorflow.keras.backend as K

K.clear_session()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(xtr.shape[1],), name='hidden'),
  tf.keras.layers.Dense(1, activation='sigmoid', name='output') #sigmoid output=binary classification
])


from tensorflow.keras import optimizers

opt = optimizers.Adam(lr=0.01) 
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit( xtr, ytr, batch_size = 3, epochs = 50, shuffle = True, validation_data=(xts,yts))

Epoch 1/50
182/182 [==============================] - 1s 3ms/step - loss: 0.1292 - accuracy: 0.9542 - val_loss: 0.0512 - val_accuracy: 0.9854
Epoch 2/50
182/182 [==============================] - 0s 2ms/step - loss: 0.0804 - accuracy: 0.9725 - val_loss: 0.0572 - val_accuracy: 0.9854
Epoch 3/50
182/182 [==============================] - 0s 2ms/step - loss: 0.0787 - accuracy: 0.9725 - val_loss: 0.0526 - val_accuracy: 0.9854
Epoch 4/50
182/182 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9725 - val_loss: 0.0475 - val_accuracy: 0.9854
Epoch 5/50
182/182 [==============================] - 0s 2ms/step - loss: 0.0708 - accuracy: 0.9780 - val_loss: 0.0517 - val_accuracy: 0.9854
Epoch 6/50
182/182 [==============================] - 1s 3ms/step - loss: 0.0689 - accuracy: 0.9780 - val_loss: 0.0473 - val_accuracy: 0.9854
Epoch 7/50
182/182 [==============================] - 1s 3ms/step - loss: 0.0681 - accuracy: 0.9780 - val_loss: 0.0457 - val_accuracy: 0.9854
Epoch 

In [25]:
d = dice_ml.Data(dataframe=df, continuous_features=[],outcome_name='class')

In [27]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=model, backend=backend)

In [28]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [30]:
df[0]

array([ 0.88360294,  0.28265077,  0.25456756,  0.01051806,  0.34102969,
        1.69443837,  1.02372986,  2.02135926, -0.35586384])

In [31]:
# query instance in the form of a dictionary; keys: feature name, values: feature value


query_instance = {'thick':0.88360294, 
                  'size':0.28265077, 
                  'shape':0.25456756, 
                  'marg':0.01051806, 
                  'cell_size':0.34102969,
                  'bare': 1.69443837, 
                  'chrom':1.02372986, 
                  'normal': 45,
                  'mit':-0.35586384}


In [36]:
print(xtr.shape,xts.shape,ytr.shape,yts.shape)

(546, 9) (137, 9) (546,) (137,)


In [32]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,728], In[1]: [9,10] [Op:MatMul]